<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/deeplearning.ai/tf/load_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Images

We will explore loading and preprocessing an image dataset using three different methods.

1. High-level Keras preprocessing utilities and layers.
2. Creating input pipelines using tf.data.
3. Using dataset from the tf repository.

## Setup

In [2]:
import os
import PIL
import PIL.Image
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
tf.__version__

'2.3.0'